In [1]:
# !git clone https://github.com/pjreddie/darknet/tree/master/cfg

Cloning into 'cfg'...
fatal: repository 'https://github.com/pjreddie/darknet/tree/master/cfg/' not found


In [3]:
!git clone https://github.com/pjreddie/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5955, done.
remote: Total 5955 (delta 0), reused 0 (delta 0), pack-reused 5955
Receiving objects: 100% (5955/5955), 6.37 MiB | 14.53 MiB/s, done.
Resolving deltas: 100% (3932/3932), done.


In [2]:
import darknet

ModuleNotFoundError: No module named 'darknet'

In [ ]:
from djitellopy import Tello
import cv2
import darknet
import time

# YOLO 설정
net, class_names, class_colors = darknet.load_network(
    "your_cfg_file.cfg",
    "your_data_file.data",
    "your_weights.weights",
    batch_size=1
)

# Tello 드론 초기화 및 연결
drone = Tello()
drone.connect()

# 드론 이륙
drone.takeoff()
time.sleep(2)

# 비디오 스트림 시작
drone.streamon()

while True:
    frame = drone.get_frame_read().frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detections = darknet.detect_image(net, class_names, frame_rgb, thresh=0.5)
    
    for label, confidence, bbox in detections:
        if label == "face":  # 얼굴을 탐지했을 경우만 처리
            x, y, w, h = darknet.bbox2points(bbox)
            cv2.rectangle(frame, (x, y), (x+w, y+h), class_colors[label], 2)
            
            # 얼굴의 중심점 계산
            center_x = x + w // 2
            center_y = y + h // 2
            
            # 화면의 중심점 계산
            frame_center_x = frame.shape[1] // 2
            frame_center_y = frame.shape[0] // 2
            
            # 드론 조정
            if center_x < frame_center_x - 30:  # 얼굴이 화면 중심 왼쪽에 있으면
                drone.move_left(20)
            elif center_x > frame_center_x + 30:  # 얼굴이 화면 중심 오른쪽에 있으면
                drone.move_right(20)
            
            if center_y < frame_center_y - 30:  # 얼굴이 화면 중심 위쪽에 있으면
                drone.move_up(20)
            elif center_y > frame_center_y + 30:  # 얼굴이 화면 중심 아래쪽에 있으면
                drone.move_down(20)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

drone.land()
drone.streamoff()
cv2.destroyAllWindows()
